## 重新考察全连接层

&emsp;&emsp;将输入和输出变形为矩阵(宽度，高度)。对应的，我们可以将权重变形为$4-D$的张量, 从$(h, w)$到$(h^{\prime}, w^{\prime})$。

&emsp;&emsp;那么我们的输出可以表示为$h_{i, j}$:

$$
h_{i, j}=\sum_{k l} w_{i, j, k, l} x_{k, l}=\sum_{a b} v_{i, j, a, b} x_{i+a, j+b}
$$

&emsp;&emsp;其中$x_{k, l}$是我们的输入, 然后对$k, l$求和。之后对下标做一些变换: $v_{i,j,a,b} = w_{i, j, i+a, j+b}$, 来引出卷积的做法, 其中$v$是$w$的重新索引:。

### 平移不变性

&emsp;&emsp;假设$x$的位置的变换，这将导致权重$v$值的变换。因此想要实现平移不变性，$v$不应该依赖于$(i, j)$，解决方法是$v_{i,j,a,b}=v_{a, b}$:

$$
h_{i,j} = \sum_{a,b} v_{a,b} x_{i+a, j+b}
$$

&emsp;&emsp;这就是2维卷积。

### 局部性

&emsp;&emsp;当评估$h_{i,j}$时，我们不应该用远离$x_{i,j}$的参数。解决方案是：当$|a|, |b| > \Delta$时，使得$v_{a, b}=0$:

$$
h_{i, j}=\sum_{a=-\Delta}^{\Delta} \sum_{b=-\Delta}^{\Delta} v_{a, b} x_{i+a, j+b}
$$



## 卷积层



<img src="../images/07-juanji.png" width="50%">

&emsp;&emsp;上图中$19$的计算方式如下: $0 \times 0 + 1 \times 1 + 3 \times 2 + 4 \times 3 = 19$。

&emsp;&emsp;假设我们的

1. 输入为$X$, 它的维度为$n_{h} \times n_{w}$；
2. 核为$W$, 它的纬度为$k_{h} \times k_{w}$,；
3. 偏差$b \in \mathbf{R}$。
4. 输出为$Y$, 纬度为: $(n_{h} - k_{h} + 1) \times (n_{w} - k_{w} + 1)$。

&emsp;&emsp;其中的$W$和$b$是可学习参数。

&emsp;&emsp;卷积层是将输入和核矩阵进行交叉相关计算，再加上偏移后得到输出。

### 互相关运算

In [1]:
import torch
def corr2d(Image, Kernel):
    N_h, N_w = Image.shape  # 获取到图像的高和宽。
    K_h, K_w = Kernel.shape  # 获取到核的高和宽。
    Y = torch.zeros((N_h - K_h + 1, N_w - K_w + 1))  # 输出结果的维度。这里步长是为1的。
    for row in range(Y.shape[0]):
        for col in range(Y.shape[1]):
            Y[row, col] = (Image[row : row + K_h, col : col + K_w] * Kernel).sum()
    return Y

In [2]:
Image = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
Kernel = torch.tensor([[0, 1], [2, 3]])
print(corr2d(Image, Kernel))

tensor([[19., 25.],
        [37., 43.]])


### 二维卷积

In [3]:
import torch.nn as nn
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

&emsp;&emsp;基于上述所实现的二维卷机，我们来实现一个简单的应用。检测图像中不同颜色的边缘。

In [4]:
X = torch.ones(6, 8)
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [5]:
Kernel = torch.tensor([[1.0, -1.0]])

&emsp;&emsp;基于$[1.0, -1.0]$的这样一个`kernel`，输出`Y`中的`1`代表从白色到黑色的边缘，`-1`表示从黑色到白色的边缘。

In [6]:
Y = corr2d(X, Kernel)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

### 学习由X生成Y的卷积核

In [7]:
# 第一个参数1表示输入的通道，第二个参数表述输出通道数也是1。
conv2d = nn.Conv2d(1, 1, kernel_size = (1, 2), bias = False)

X = X.reshape((1, 1, 6, 8))  # 需要增加两个维度，一个是通道位，一个是批量大小位。
Y = Y.reshape((1, 1, 6, 7))

for i in range(10):
    Y_hat = conv2d(X)
    loss = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    loss.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    
    if (i + 2) % 2 == 0:
        print("batch {}, loss {}".format(i, loss.sum()))

batch 0, loss 10.075425148010254
batch 2, loss 2.4747474193573
batch 4, loss 0.7364716529846191
batch 6, loss 0.25515487790107727
batch 8, loss 0.0967092365026474


&emsp;&emsp;查看一下学习结果:

In [8]:
conv2d.weight.data

tensor([[[[ 0.9663, -1.0163]]]])

## 填充和步幅

&emsp;&emsp;填充和步幅是控制卷积层输出大小的两个参数。输出图片的大小为$(n_{h} - k_{h} + 1) \times (n_{w} - k_{w} + 1)$, 因此卷积之后输出图片会越变越小，而深度学习考虑的是如何用更深的网络来训练模型，所以想要更深的网络的话，我们需要做填充：**在输入周围添加额外的行和列**。

&emsp;&emsp;假设填充的为$p_{h}$行和$p_{w}$列，那么此时输出图片的大小为：$(n_{h} - k_{h} + p_{h} + 1) \times (n_{w} - k_{w} + p_{w} + 1)$。

&emsp;&emsp;通常来说取，$p_{h}=k_{h} - 1$, $p_{w} = k_{w} - 1$，此时的输入输出维度是相等的：

- 当$k_{h}$为奇数：在上下两侧填充$p_{h} / 2$。
- 当$k_{h}$为偶数：在上侧填充$「p_{h} / 2$，下侧填充$p_{h} / 2」$。

&emsp;&emsp;对于填充来说，是希望可以控制图片不要变小地太快，但是对于一张很大的图片来说，我们希望它能够快速变小的话，就可以通过步幅来进行调整。不然的话，网络可能会太深，需要大量的计算得出较小的输出。

&emsp;&emsp;给定高度$s_{h}$和宽度$s_{w}$的步幅，输出形状是：$((n_{h} - k_{h} + p_{h}) / s_{h} + 1) \times ((n_{w} - k_{w} + p_{w}) / s_{w} + 1)$。

In [3]:
import torch
import torch.nn as nn

def comp_conv2d(conv2d, x):
    x = x.reshape((1, 1) + x.shape)  # 添加批量数和通道数。
    y = conv2d(x)  # 应用卷积。
    return y.reshape(y.shape[2:])  # 去除掉批量数和通道数。

In [4]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)  # 输入输出通道数都为1。padding=1的话，会在上下左右各填充一行。
x = torch.rand(size=(8, 8))  
comp_conv2d(conv2d, x).shape  # 输出大小和输入大小是一样的了。

torch.Size([8, 8])

&emsp;&emsp;当核的大小变为(5, 3)的时候，只有当上下填充2行，左右填充1列的时候，输出大小才会一样。

In [5]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, x).shape

torch.Size([8, 8])

&emsp;&emsp;将高度和宽度的步幅设置为2的时候:

In [7]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, x).shape

torch.Size([4, 4])

&emsp;&emsp;一个稍微复杂一点的例子:

In [8]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, x).shape

torch.Size([2, 2])

## 问答

1. 核大小、填充、步幅这几个超参数的重要程度排序是怎样的？

- 一般来说填充就是取核的大小减去1，使得输入输出的大小一样。
- 步幅通常选择为1，通常计算量很大的话，我们不会选择步幅为1的情况。

2. 卷积核的边长为什么一般取奇数？

- 卷积核的边长取奇数的原因在于填充的时候能够使得上下填充是对称的。

## 卷积层里的多输入多输出通道

&emsp;&emsp;通道数通常是大家通常会仔细去设计的超参数:

- **多个输入通道**:

&emsp;&emsp;图像通常是多个输入通道的，假设我们的

1. 输入为$X$, 它的维度为: $c_{i} \times n_{h} \times n_{w}$；
2. 核为$W$, 它的纬度为: $c_{i} \times k_{h} \times k_{w}$；
3. 偏差$b \in \mathbf{R}$。
4. 输出为$Y$, 纬度为: $m_{h} \times m_{w}$。

&emsp;&emsp;也就是说，输出是一个单通道的。每一个通道的对应元素相加，得到最终的单通道的输出。

- **多个输出通道**:

&emsp;&emsp;我们可以用多个三维卷积核，每个核生成一个输出通道。

1. 输入为$X$, 它的维度为: $c_{i} \times n_{h} \times n_{w}$；
2. 核为$W$, 它的纬度为: $c_{o} \times c_{i} \times k_{h} \times k_{w}$；
3. 偏差$b \in \mathbf{R}$。
4. 输出为$Y$, 纬度为: $c_{o} \times m_{h} \times m_{w}$。

- **多个输入和输出通道**

&emsp;&emsp;每个输出通道可以识别特定模式。输入通道核识别并组合输入中的模式。

- $1 \times 1$卷积层

&emsp;&emsp;$k_{h} = k_{w} = 1$是一个受欢迎的选择，它不识别空间模式，只是融合通道。

- **二维卷积层通用情况**:

&emsp;&emsp;我们可以用多个三维卷积核，每个核生成一个输出通道。

1. 输入为$X$, 它的维度为: $c_{i} \times n_{h} \times n_{w}$；
2. 核为$W$, 它的纬度为: $c_{o} \times c_{i} \times k_{h} \times k_{w}$；
3. 偏差$c_{o} \times c_{i}$。
4. 输出为$Y$, 纬度为: $c_{o} \times m_{h} \times m_{w}$。

In [17]:
import torch

def corr2d(Image, Kernel):
    N_h, N_w = Image.shape  # 获取到图像的高和宽。
    K_h, K_w = Kernel.shape  # 获取到核的高和宽。
    Y = torch.zeros((N_h - K_h + 1, N_w - K_w + 1))  # 输出结果的维度。这里步长是为1的。
    for row in range(Y.shape[0]):
        for col in range(Y.shape[1]):
            Y[row, col] = (Image[row : row + K_h, col : col + K_w] * Kernel).sum()
    return Y

- **实现多输入通道的互相关计算**:

In [18]:
def corr2d_multi_in(X, K):
    return sum(corr2d(x, k) for x, k in zip(X, K))

&emsp;&emsp;验证互相关运算的输出:

In [19]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

- **计算多个通道的输出的互相关函数**:

In [20]:
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)  # 在0这个维度上进行stack。

In [21]:
K = torch.stack((K, K+1, K+2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [22]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## $1 \times 1$卷积等价于全连接

In [23]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [25]:
X = torch.normal(0, 1, ((3, 3, 3)))
K = torch.normal(0, 1, ((2, 3, 1, 1)))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

## 问题

- 输入和输出的通道数如果不变的话，通道数一般也不变；输入和输出的通道数减半的时候，输出的通道数通常会加一倍，也就是空间信息压缩了，把提取的信息在更多的通道里面存储下来。

1. **网络越深，padding 0越多，是否会影响性能？**

&emsp;&emsp;计算性能增加了一点点，对于模型性能是不会影响的，因为0对于卷积是没有多少影响的。

2. **每个通道的卷积核都不一样吗？同一层不同通道的卷积核大小必须一样吗？**

&emsp;&emsp;每个通道的卷积核是不一样的，同一层不同通道的卷积核大小必须一样。

3. **计算卷积时，bias的有无，对结果影响大吗？bias的作用怎么理解？**

&emsp;&emsp;bias是有一些作用的，但是它的作用会变得越来越低。比如像数据的均值不为0的时候，bias就可以去等价于均值的负数。

## 池化层

&emsp;&emsp;卷积层对于位置信息是非常敏感的，而实际的图像因为照明，物体的位置，比例，外观等因图像而异，所以我们需要一定的平移不变性。

### 二维最大池化

&emsp;&emsp;每次返回二维窗口中的最大值。因此最大池化层会允许输入的元素发生小小的偏移，并且会有一定的模糊效果，但是它没有可学习的参数，输出通道数会等于输入通道数。

### 平均池化层

&emsp;&emsp;把最大的那个操作子变成平均。最大池化层获取的是每个窗口中最强的那个信号，平均池化层的信号强化会弱很多，但是会有一个比较柔和化的效果。

### 小结

&emsp;&emsp;它会缓解卷积层带来的位置敏感性。同样有窗口大小、填充、和步幅作为超参数。

## 代码实现

&emsp;&emsp;实现池化层的正向传播:

In [26]:
import torch
import torch.nn as nn

def pool2d(X, pool_size, mode="max"):
    x_h, x_w = X.shape
    p_h, p_w = pool_size
    Y = torch.zeros((x_h - p_h + 1, x_w - p_w + 1))  # 创建输出
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == "max":
                Y[i, j] = X[i:i+p_h, j:j+p_w].max()
            elif mode == "avg":
                Y[i, j] = X[i:i+p_h, j:j+p_w].mean()
    return Y

In [27]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])

In [28]:
# 验证最大池化
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [29]:
# 验证平均池化
pool2d(X, (2, 2), mode="avg")

tensor([[2., 3.],
        [5., 6.]])

&emsp;&emsp;`torch`中的代码实现:

In [30]:
X = torch.arange(16, dtype=torch.float32).reshape(1, 1, 4, 4)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

&emsp;&emsp;深度学习中的步幅与池化窗口相同(torch框架是这样的)，也就是移动的时候，下一个池化区域与上一个池化区域没有重叠部分:

In [31]:
pool2d = nn.MaxPool2d(3)  # 设定一个3 X 3的窗口
pool2d(X)

tensor([[[[10.]]]])

&emsp;&emsp;填充和步幅可以手动设定

In [32]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

&emsp;&emsp;设定一个任意大小的矩形池化窗口，并分别设定填充和步幅的高度和宽度:

In [33]:
pool2d = nn.MaxPool2d((2, 3), padding=(1, 1), stride=(2, 3))
pool2d(X)

tensor([[[[ 1.,  3.],
          [ 9., 11.],
          [13., 15.]]]])

&emsp;&emsp;池化层在每个通道上单独运算

In [34]:
X = torch.cat((X, X+1), 1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [35]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])